In [1]:
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from rich.console import Console

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time
import random
import re

In [5]:
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Ejecutar en modo sin cabeza (sin interfaz gráfica)
chrome_options.add_argument("--no-sandbox") # Mejora por el No - Sandbox que inpone google
chrome_options.add_argument("--disable-dev-shm-usage") # Mejora uso memoria en contendores.
chrome_options.add_argument("--incognito") 
chrome_options.add_argument("--disable-gpu")  # Puede ayudar si hay problemas con los gráficos
chrome_options.add_argument("start-maximized")  # Iniciar con la ventana maximizada
chrome_options.add_argument("--disable-extensions")  # Desactivar extensiones
chrome_options.add_argument("--disable-popup-blocking")  # Desactivar el bloqueo de pop-ups

In [6]:
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.itsitio.com/")